In [2]:
!pip3 uninstall mecab --yes
!pip3 uninstall mecab-python3 --yes

In [3]:
!brew install --build-from-source mecab
!ARCHFLAGS='-arch arm64' pip3 install --no-binary :all: --compile --use-pep517 --no-cache-dir --force mecab-python3
!ARCHFLAGS='-arch arm64' pip3 install --no-binary :all: --compile --use-pep517 --no-cache-dir --force mecab

To reinstall 0.996, run:
  brew reinstall mecab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mecab-python3: filename=mecab_python3-1.0.8-cp312-cp312-macosx_11_0_arm64.whl size=39714 sha256=665cdacebb81abd7009feaeede0f8144759729fccabac577dd4832681ea5771e
  Stored in directory: /private/var/folders/zj/mbm_nh6j6bn7bqqd10llb1lm0000gn/T/pip-ephem-wheel-cache-_kbhw03x/wheels/58/73/51/e2f5f2305bc33c57b0e03dc8cbbbc41b0e640273c31a20befa
Successfully built mecab-python3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mecab: filename=mecab-0.996.3-cp312-cp312-macosx_11_0_arm64.whl siz

In [5]:
import json
import MeCab
import demoji
import re
from stop_words import stop_words
import gensim, logging

ImportError: dlopen(/usr/local/Caskroom/miniforge/base/envs/thesis/lib/python3.12/site-packages/MeCab/_MeCab.cpython-312-darwin.so, 0x0002): symbol not found in flat namespace '__ZN5MeCab11createModelEPKc'

cleaning and tokenizing

In [ ]:
# cleaning and tokenizing

mt = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
file = open('test-tweets.txt', 'r')

# we'll just test on a few tweets for now
tweets = []
for i in range(100):
    # load tweet, convert to py dict, access content
    tweet_json = file.readline()
    tweet_py = json.loads(tweet_json)
    tweet_text = tweet_py['rawContent'] # note: need other prop for over 140 char?

    # clean tweet content
    remove_emojis = demoji.replace(tweet_text, "")
    remove_more_emojis = re.sub("([\uD83E-\uD83E])+", "", remove_emojis)
    remove_newlines = re.sub("(\n)+", "", remove_more_emojis)
    remove_usernames = re.sub("@([a-zA-Z0-9_]+)", "", remove_newlines)
    remove_hashtags = re.sub("#([a-zA-Z0-9_ぁ-んァ-ン一-龠]+)", "", remove_usernames)
    remove_links = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", remove_hashtags)
    remove_punc = re.sub("([-.,;\"\'!?~@#$%^&*():\{\}\[\]\/\\\\]+)", "", remove_links)
    remove_jp_punc = re.sub("([\uFF01-\uFF0F\uFF1A-\uFF20\uFF3B-\uFF40\uFF5B-\uFF65\uFF9E-\uFFEE\u3000-\u303F]+)", "", remove_punc)
    remove_geo_shapes = re.sub("([\u25A0-\u25FF])+", "", remove_jp_punc)
    remove_misc_symbols = re.sub("([\u2600-\u26FF])+", "", remove_geo_shapes)

    # mecab tokenization
    parsed = mt.parseToNode(remove_misc_symbols)
    components = []
    while parsed:
        components.append(parsed.surface)
        parsed = parsed.next
    components = [token for token in components if not token in stop_words]
    tweets.append(components)

<>:20: SyntaxWarning: invalid escape sequence '\('
<>:21: SyntaxWarning: invalid escape sequence '\{'
<>:20: SyntaxWarning: invalid escape sequence '\('
<>:21: SyntaxWarning: invalid escape sequence '\{'
/var/folders/zj/mbm_nh6j6bn7bqqd10llb1lm0000gn/T/ipykernel_66914/4078735437.py:20: SyntaxWarning: invalid escape sequence '\('
  remove_links = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", remove_hashtags)
/var/folders/zj/mbm_nh6j6bn7bqqd10llb1lm0000gn/T/ipykernel_66914/4078735437.py:21: SyntaxWarning: invalid escape sequence '\{'
  remove_punc = re.sub("([-.,;\"\'!?~@#$%^&*():\{\}\[\]\/\\\\]+)", "", remove_links)
/var/folders/zj/mbm_nh6j6bn7bqqd10llb1lm0000gn/T/ipykernel_66914/4078735437.py:20: SyntaxWarning: invalid escape sequence '\('
  remove_links = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", remove_hashtags)
/var/folders/zj/mbm_nh6j6bn7bqqd10llb1lm0000gn/T/ipykernel_66914/4078735437.

NameError: name 'MeCab' is not defined

we'll just test on a few tweets for now

In [ ]:
tweets = []
for i in range(100):
    # load tweet, convert to py dict, access content
    tweet_json = file.readline()
    tweet_py = json.loads(tweet_json)
    tweet_text = tweet_py['rawContent'] # note: need other prop for over 140 char?

    # clean tweet content
    remove_emojis = demoji.replace(tweet_text, "")
    remove_more_emojis = re.sub("([\uD83E-\uD83E])+", "", remove_emojis)
    remove_newlines = re.sub("(\n)+", "", remove_more_emojis)
    remove_usernames = re.sub("@([a-zA-Z0-9_]+)", "", remove_newlines)
    remove_hashtags = re.sub("#([a-zA-Z0-9_ぁ-んァ-ン一-龠]+)", "", remove_usernames)
    remove_links = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", remove_hashtags)
    remove_punc = re.sub("([-.,;\"\'!?~@#$%^&*():\{\}\[\]\/\\\\]+)", "", remove_links)
    remove_jp_punc = re.sub("([\uFF01-\uFF0F\uFF1A-\uFF20\uFF3B-\uFF40\uFF5B-\uFF65\uFF9E-\uFFEE\u3000-\u303F]+)", "", remove_punc)
    remove_geo_shapes = re.sub("([\u25A0-\u25FF])+", "", remove_jp_punc)
    remove_misc_symbols = re.sub("([\u2600-\u26FF])+", "", remove_geo_shapes)

    # mecab tokenization
    parsed = mt.parseToNode(remove_misc_symbols)
    components = []
    while parsed:
        components.append(parsed.surface)
        parsed = parsed.next
    components = [token for token in components if not token in stop_words]
    tweets.append(components)

print(tweets)

word2vec

set up logging

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

train word2vec

In [ ]:
model = gensim.models.Word2Vec(tweets, min_count=3)

check similarity given by trained model

In [ ]:
sim = model.wv.most_similar('今日')
print(sim)